In [8]:
def running_avg():
    print('Start running average')
    number = 0
    numbers = 0
    total = 0
    while True:
        total += yield number
        numbers += 1
        avg = total/numbers
        print('The current average is:', avg)

In [18]:
avger = running_avg()
avger.send(None)
avger.send(10)
avger.send(6)
avger.send(29)
avger.close()
avger.send(29)


Start running average
The current average is: 10.0
The current average is: 8.0
The current average is: 15.0


StopIteration: 

In [22]:
def finite_coroutine():
    print('Coroutine starts')
    try:
        value = yield
    except GeneratorExit:
        print('Being closed')
        return
    print('Receive value {} and stops'.format(value))
    
coro = finite_coroutine()
coro.send(None)
coro.close()

Coroutine starts
Being closed


In [18]:

def proxy():
    print('proxy is started')
    result = yield from another_proxy()
    print('Resume to proxy with result', result)
    yield result
    print ('Resume to proxy again')
    
    
def another_proxy():
    print('another_proxy is started')
    result = yield from real_gen()
    print('Resume to another_proxy with result', result)
    print("I don't want to return yet, waiting here for value")
    own_value = yield
    print("Resume again to another_proxy with result:", own_value)
    return result

def real_gen():
    print("I'm the real deal")
    while True:
        result = yield
        if result is None:
            return 'inner'
        
        print('Getting value from client:', result)

prx = proxy()
next(prx)
result = prx.send('value')
print(result)
result = prx.send('Another value')
prx.throw(ValueError)
result = prx.send(None)
result = prx.send('value again')


proxy is started
another_proxy is started
I'm the real deal
Getting value from client: value
None
Getting value from client: Another value


ValueError: 

In [19]:
def normal_gen():
    print("start gen")
    yield
    print("result gen")
    print("do something")
    
gen = normal_gen()
next(gen)
gen.throw(ValueError)
next(gen)

start gen


ValueError: 

In [20]:
from concurrent import futures
from time import sleep

def lazy_task(msg):
    print(futures.thread.threading.current_thread())
    print('Going to sleep first')
    sleep(2)
    print('Printing message:', msg)
    sleep(2)
    print('Exiting,', msg)
    
    return msg + ' is completed'

def show(fs):
    print(futures.thread.threading.current_thread())
    print(fs)
    for future in fs:
        print('Trying to print result from future:', future)
        print(future.result())
        
def show_as_completed(fs):
    print(futures.thread.threading.current_thread())
    print(fs)
    for future in futures.as_completed(fs):
        print('>>>>>>>>>>>Trying to print result from future:', future)
        print(future.result())
        
msg = ['Hello', 'There', 'I', 'Love', 'You']
fs = []
print(futures.thread.threading.current_thread())
with futures.ThreadPoolExecutor(max_workers=6) as executor:
    for m in msg:
        future = executor.submit(lazy_task, m)
        print(m, '- Future submitted:', future)
        fs.append(future)
        
    executor.submit(show_as_completed, fs)


<_MainThread(MainThread, started 21664)>
<Thread(ThreadPoolExecutor-15_0, started daemon 22632)>
Going to sleep first
Hello - Future submitted: <Future at 0x1ce3edabf70 state=running>
<Thread(ThreadPoolExecutor-15_1, started daemon 7112)>There - Future submitted: <Future at 0x1ce3e7e5f70 state=running>
<Thread(ThreadPoolExecutor-15_2, started daemon 22744)>
Going to sleep first
I
Going to sleep first
 - Future submitted: <Future at 0x1ce3e8a5a60 state=running>
<Thread(ThreadPoolExecutor-15_3, started daemon 8096)>Love - Future submitted: <Future at 0x1ce3ef398b0 state=running>

Going to sleep first
<Thread(ThreadPoolExecutor-15_4, started daemon 10620)>
Going to sleep first
You - Future submitted: <Future at 0x1ce3ea72e50 state=running>
<Thread(ThreadPoolExecutor-15_5, started daemon 19288)>
[<Future at 0x1ce3edabf70 state=running>, <Future at 0x1ce3e7e5f70 state=running>, <Future at 0x1ce3e8a5a60 state=running>, <Future at 0x1ce3ef398b0 state=running>, <Future at 0x1ce3ea72e50 state=r

In [11]:
futures.thread.threading.current_thread().

<_MainThread(MainThread, started 21664)>